In [20]:
#-----01 within user----------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import sys,os
from seglearn.transform import SegmentX
from seglearn.feature_functions import maximum,minimum
from sklearn.preprocessing import normalize,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import StratifiedKFold
import time


In [21]:
# --------starts here-------
skip_list = [0,1,3,6,8]
#label
y=[]
# for k in range(10):#num of participants

for i in range(14):
    if i in skip_list:
        continue
    for j in range(30):
        y.append(i)

y_train = y*9
y_test = y[:]
print(len(y))
skip_len = len(y)
tsfresh_feature = pd.read_csv("../data/final_filtered2400.csv")
tsfresh_feature = tsfresh_feature.drop(["id"],axis=1)
selected_list = []
for i in range(10):
    for j in range(14):
        if(j in skip_list):continue
        for k in range(30):
            selected_list.append(i*420+j*30+k)
tsfresh_feature = tsfresh_feature.iloc[selected_list]
print(tsfresh_feature.shape)

270
(2700, 463)


In [ ]:
# tsfresh_feature

In [ ]:

# import pickle

# # define a list of places
# cjc_feature_list = list(tsfresh_feature.columns)

# with open('cjc_feature_list.data', 'wb') as filehandle:
#     # store the data as binary data stream
#     pickle.dump(cjc_feature_list, filehandle)

In [22]:
#------ adding features from Shutong ------#
# fftfeatures1 = np.loadtxt(fname = "data/22gesture.txt",delimiter=',')
# fftfeatures = np.loadtxt(fname = "data/qstSpectro.txt",delimiter=',')
# fftfeatures = np.concatenate((fftfeatures1,fftfeatures2),axis=0)
fftfeatures = np.loadtxt(fname = "../data/fftFeature2400.txt",delimiter=',')
print(fftfeatures.shape)
_len = len(fftfeatures[:][0])
fft_df = pd.DataFrame(data=fftfeatures[:],    # values
              index=[i for i in range(4200)],    # 1st column as index
              columns=["f"+str(i) for i in range(_len)])

fft_df = fft_df.iloc[selected_list]
print(fft_df.shape)
features_combine = fft_df.join(tsfresh_feature)
features_combine.shape

(4200, 100)
(2700, 100)


(2700, 563)

In [10]:
features_combine[0:0]

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,"v__fft_coefficient__coeff_92__attr_""imag""",v__sum_of_reoccurring_data_points,"v__fft_coefficient__coeff_91__attr_""imag""","v__fft_coefficient__coeff_10__attr_""angle""","v__fft_coefficient__coeff_8__attr_""imag""",v__ratio_beyond_r_sigma__r_1.5,"v__fft_coefficient__coeff_76__attr_""imag""",v__partial_autocorrelation__lag_8,v__ratio_beyond_r_sigma__r_1,"v__fft_coefficient__coeff_9__attr_""angle"""


In [24]:
maxyhat_train=[0,0,0,0,0,0,0,0,0,0]
maxyhat_test=[0,0,0,0,0,0,0,0,0,0]
temp = 0
feature_list = []
train_results = [[],[],[],[],[],[],[],[],[],[]]
test_results = [[],[],[],[],[],[],[],[],[],[]]
# for i in range(10,20):
seconds = time.time()
for people in range(10):
    x_test = features_combine[people*skip_len:(people+1)*skip_len]
    x_train = features_combine[0*skip_len:people*skip_len]
    x_train = pd.concat([x_train,features_combine[(people+1)*skip_len:10*skip_len]])
    for j in range(0,10):
#         x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=i, train_size=0.9, test_size=0.1)
        clf = RandomForestClassifier(n_estimators=100, max_depth=30, random_state=j)
        clf.fit(x_train, y_train)
        temp = clf.score(x_train, y_train)
        train_results[people].append(temp)
        if (temp > maxyhat_train[people]).any():
            maxyhat_train[people] = temp
        temp = clf.score(x_test, y_test)
        test_results[people].append(temp)
        feature_importances = pd.DataFrame(clf.feature_importances_,
                               index = x_train.columns,
                                columns=['importance']).sort_values('importance',ascending=False).sort_values('importance',ascending=False)
        if (maxyhat_test[people] < temp).any():
            maxyhat_test[people] = temp
            for feature in feature_importances.index.tolist()[:50]:
                if not feature in feature_list:
                    feature_list.append(feature)
    seconds = time.time()-seconds
    print(seconds)
    seconds = time.time()
print(maxyhat_train)
print(maxyhat_test)

38.04583215713501
37.24566149711609
33.09847116470337
31.99525022506714
39.56912660598755
3045.931287050247
32.38419270515442
32.65915775299072
31.517261028289795
7098.407863140106
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[0.8814814814814815, 0.8962962962962963, 0.825925925925926, 0.7074074074074074, 0.8777777777777778, 0.762962962962963, 0.7962962962962963, 0.9259259259259259, 0.6925925925925925, 0.8037037037037037]


In [15]:
mylist = []
for i in range(10):
    mylist.append(np.max(test_results[i]))
print(mylist)
print(np.mean(mylist))
print(np.std(mylist))


[0.8814814814814815, 0.7592592592592593, 0.8222222222222222, 0.762962962962963, 0.8925925925925926, 0.8222222222222222, 0.6259259259259259, 0.8740740740740741, 0.6407407407407407, 0.8111111111111111]
0.7892592592592592
0.08908081132602048


In [ ]:
feature_list
# import pickle


# with open('feature_list.data', 'wb') as filehandle:
#     # store the data as binary data stream
#     pickle.dump(feature_list, filehandle)

In [ ]:
# y = y
# maxyhat_train=[0,0,0,0,0,0,0,0,0,0]
# maxyhat_test=[0,0,0,0,0,0,0,0,0,0]
# temp = 0
# train_results = [[],[],[],[],[],[],[],[],[],[]]
# test_results = [[],[],[],[],[],[],[],[],[],[]]
# # for i in range(10,20):
# seconds = time.time()
# for people in range(10):
#     #get each user
#     X = features_combine[people*420:(people+1)*420]
#     for i in range(10,20):
#         for j in range(10,20):
#             x_train, x_test, y_train, y_test = train_test_split(X[feature_list], y, random_state=i, train_size=0.5, test_size=0.5)
            
#             clf = RandomForestClassifier(n_estimators=100, max_depth=30, random_state=j)
#             clf.fit(x_train, y_train)
#             temp = clf.score(x_train, y_train)
#             train_results[people].append(temp)
#             if (temp > maxyhat_train[people]).any():
#                 maxyhat_train[people] = temp
#             temp = clf.score(x_test, y_test)
#             test_results[people].append(temp)
#             feature_importances = pd.DataFrame(clf.feature_importances_,
#                                    index = x_train.columns,
#                                     columns=['importance']).sort_values('importance',ascending=False).sort_values('importance',ascending=False)
#             if (maxyhat_test[people] < temp).any():
#                 maxyhat_test[people] = temp
              
#     seconds = time.time()-seconds
#     print(seconds)
#     seconds = time.time()
# print(maxyhat_train)
# print(maxyhat_test)

In [ ]:
# mylist = []
# for i in range(10):
#     mylist.append(np.max(test_results[i]))
# print(mylist)
# np.mean(mylist)


In [ ]:
# from matplotlib.legend_handler import HandlerLine2D
# line1, = plt.plot(np.arange(100), train_results,'b', label='Train AUC')
# line2, = plt.plot(np.arange(100), test_results,'r', label='Test AUC')
# plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
# plt.ylabel('AUC score')
# plt.xlabel('n_estimators')
# plt.show()

In [18]:

y_max=[]
for i in range(10):
    y_max.append(10+np.argmax(test_results[i]))
    print(np.argmax(test_results[i]))

3
3
7
9
9
3
5
5
7
4


In [19]:
y_max

[13, 13, 17, 19, 19, 13, 15, 15, 17, 14]

In [ ]:
print(np.argmax(test_results))
#删4个56

In [ ]:
#------ using ranked features ----------#
bestmodel = 0
maxyhat_test=0



fig = None
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    global fig
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]*100
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

#     print(cm)
    plt.rcParams.update({'font.size': 18})
    plt.rc('ytick', labelsize=20)
    plt.rc('xtick', labelsize=20)
    plt.rc('axes', labelsize=26)

    fig, ax = plt.subplots(figsize=(15,15))
    
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.1f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

v2_names=['button','check','hand_circle','double_tap','swipe','finger_rub',
         'finger_slide','hold','slow_swipe','pull','push','rect','finger_circle',
         'single_tap']
v2_names_sort = ['button','finger_rub','finger_slide','','finger_circle','check','rectangle','pull','','push','hold','swipe','hand_circle','tap']
projection = [0,5,12,3,11,1,2,10,8,7,9,6,4,13]
v2_names_sort = np.array(v2_names_sort)
v2_names = np.array(v2_names)

y_pred = []
#get each user
i=14
j=y_max[people]

for people in range(10):
    x_test = features_combine[people*skip_len:(people+1)*skip_len]
    x_train = features_combine[0*skip_len:people*skip_len]
    x_train = pd.concat([x_train,features_combine[(people+1)*skip_len:10*skip_len]])
    for j in range(0,100):
#         x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=i, train_size=0.9, test_size=0.1)
        clf = RandomForestClassifier(n_estimators=100, max_depth=30, random_state=j)
        clf.fit(x_train, y_train)
        temp = clf.score(x_train, y_train)
        train_results[people].append(temp)
        if (temp > maxyhat_train[people]).any():
            maxyhat_train[people] = temp
        temp = clf.score(x_test, y_test)
        test_results[people].append(temp)


X=features_combine
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=i, train_size=0.9, test_size=0.1)

clf = RandomForestClassifier(n_estimators=100, max_depth=30, random_state=j)
clf.fit(x_train, y_train)
print(clf.score(x_test,y_test))
feature_importances = pd.DataFrame(clf.feature_importances_,
                               index = x_train.columns,
                                columns=['importance']).sort_values('importance',ascending=False).sort_values('importance',ascending=False)
print(feature_importances.index.tolist()[:50])
y_pred = list(clf.predict(x_test))
#     print(list(clf.predict(x_test)))
print(y_test)   
print(y_pred)
y_test_new = []
y_pred_new = []
for i in range(len(y_test)):
    y_test_new.append(projection[y_test[i]])
    y_pred_new.append(projection[y_pred[i]])
print(y_test_new)   
print(y_pred_new)
plot_confusion_matrix(y_test_new, y_pred_new, classes=v2_names_sort, normalize=True,
          title='')
plt.show()

seconds = time.time()-seconds
#     print(seconds)
seconds = time.time()
print(maxyhat_train)
print(maxyhat_test)
fig.savefig("2400_cross.pdf", bbox_inches = "tight")

In [ ]:
from joblib import dump, load
dump(clf, '4200.joblib')

In [ ]:
clf2 = load('filename.joblib')